# PART 1

In [68]:
import open3d as o3d
import numpy as np

# Путь к файлу с облаком точек
velodyne_file = 'D:/Cluster_proj/VIPCClustering/data/KITTI/velodyne/000008.bin'

# Загрузка облака точек из файла .bin
velo_points = np.fromfile(velodyne_file, dtype=np.float32).reshape(-1, 4)
points_np = velo_points[:, :3]

# Создание облака точек
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_np)

# Отображение облака точек
o3d.visualization.draw_geometries([pcd])


In [69]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Путь к файлу с облаком точек

velodyne_file = 'D:/Cluster_proj/VIPCClustering/data/KITTI/velodyne/000008.bin'

# Загрузка облака точек из файла .bin
velo_points = np.fromfile(velodyne_file, dtype=np.float32).reshape(-1, 4)
points_np = velo_points[:, :3]

# Проверка наличия данных в облаке точек
if len(points_np) == 0:
    print("Облако точек пустое или файл не содержит данных.")
else:
    # Масштабирование данных
    scaler = StandardScaler()
    scaled_points_np = scaler.fit_transform(points_np)

    # Отсечение точек снизу по оси Z
    cutting_height = -1.2
    cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
    cut_points_np = points_np[cut_indices]
    cut_scaled_points_np = scaled_points_np[cut_indices]

    # Определение числа ближайших соседей
    n_neighbors = 1

    # Поиск ближайших соседей для каждой точки
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
    distances, indices = nbrs.kneighbors(cut_scaled_points_np)

    # Вычисление среднего расстояния до n_neighbors точек для каждой точки
    avg_distances = np.mean(distances, axis=1)

    # Определение порогового значения для удаления редких точек
    threshold_low = np.percentile(avg_distances, 1)

    # Отфильтровать точки по пороговому значению
    filtered_indices = np.where(avg_distances <= threshold_low)[0]
    filtered_cut_points_np = cut_points_np[filtered_indices]

    # Выполнение кластеризации K-means
    num_clusters = 200
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(filtered_cut_points_np)
    labels = kmeans.labels_

    # Присвоение цветов кластерам
    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

    # Создание облака точек с цветами кластеров
    kmeans_pcd = o3d.geometry.PointCloud()
    kmeans_pcd.points = o3d.utility.Vector3dVector(filtered_cut_points_np)
    kmeans_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

    # Создание ограничивающего параллелепипеда для каждого кластера
    bbox_list = []
    for label in np.unique(labels):
        if label == -1:
            continue
        cluster_points = filtered_cut_points_np[labels == label]
        bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster_points))
        bbox.color = (0, 0, 0)
        bbox_list.append(bbox)

    # Визуализация облака точек с кластеризацией K-means и ограничивающих параллелепипедов
    o3d.visualization.draw_geometries([kmeans_pcd] + bbox_list)

    # Печать кластеров с количеством точек до 200 в каждом кластере
    print("Кластеры с количеством точек до 200 в каждом кластере:")
    unique_labels, counts = np.unique(labels, return_counts=True)
    sorted_clusters = sorted(zip(unique_labels, counts), key=lambda x: x[1], reverse=True)
    for cluster_label, point_count in sorted_clusters:
        if point_count <= 200:
            print(f"Кластер {cluster_label}: {point_count} точек")
    
    # Удаление кластеров с количеством точек менее или равным 200
filtered_indices = np.ones(len(labels), dtype=bool)
for cluster_label, point_count in sorted_clusters:
    if point_count <= 200:
        filtered_indices[labels == cluster_label] = False

# Фильтрация точек с использованием индексов
filtered_points_np = filtered_cut_points_np[filtered_indices]

# Создание облака точек после удаления кластеров
filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)

# Визуализация облака точек после удаления кластеров
o3d.visualization.draw_geometries([filtered_pcd])


Кластеры с количеством точек до 200 в каждом кластере:
Кластер 2: 199 точек
Кластер 119: 191 точек
Кластер 82: 188 точек
Кластер 29: 182 точек
Кластер 86: 178 точек
Кластер 100: 170 точек
Кластер 198: 169 точек
Кластер 154: 167 точек
Кластер 76: 164 точек
Кластер 109: 163 точек
Кластер 62: 161 точек
Кластер 70: 161 точек
Кластер 131: 157 точек
Кластер 156: 157 точек
Кластер 1: 155 точек
Кластер 133: 154 точек
Кластер 24: 151 точек
Кластер 166: 141 точек
Кластер 25: 139 точек
Кластер 150: 136 точек
Кластер 152: 135 точек
Кластер 180: 132 точек
Кластер 30: 129 точек
Кластер 89: 127 точек
Кластер 93: 127 точек
Кластер 122: 126 точек
Кластер 171: 126 точек
Кластер 144: 124 точек
Кластер 22: 123 точек
Кластер 26: 122 точек
Кластер 160: 122 точек
Кластер 179: 121 точек
Кластер 125: 118 точек
Кластер 58: 117 точек
Кластер 53: 113 точек
Кластер 71: 113 точек
Кластер 117: 111 точек
Кластер 34: 109 точек
Кластер 13: 98 точек
Кластер 192: 98 точек
Кластер 35: 95 точек
Кластер 44: 94 точек
Кластер

In [70]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Заменяем переменную velodyne_file на filtered_points_np
points_np = filtered_points_np  # filtered_points_np должна быть определена заранее

# Проверка наличия данных в облаке точек
if len(points_np) == 0:
    print("Облако точек пустое.")
else:
    # Масштабирование данных
    scaler = StandardScaler()
    scaled_points_np = scaler.fit_transform(points_np)

    # Отсечение точек снизу по оси Z
    cutting_height = -1.2
    cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
    cut_points_np = points_np[cut_indices]
    cut_scaled_points_np = scaled_points_np[cut_indices]

    # Определение числа ближайших соседей
    n_neighbors = 1

    # Поиск ближайших соседей для каждой точки
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
    distances, indices = nbrs.kneighbors(cut_scaled_points_np)

    # Вычисление среднего расстояния до n_neighbors точек для каждой точки
    avg_distances = np.mean(distances, axis=1)

    # Определение порогового значения для удаления редких точек
    threshold_low = np.percentile(avg_distances, 1)

    # Отфильтровать точки по пороговому значению
    filtered_indices = np.where(avg_distances <= threshold_low)[0]
    filtered_points_np = cut_points_np[filtered_indices]

    # Вычисление центра масс облака точек
    center = np.mean(filtered_points_np, axis=0)

    # Создание красной точки в центре облака точек
    center_point = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)
    center_point.paint_uniform_color([1.0, 0.0, 0.0])
    center_point.translate(center)

    # Выполнение кластеризации K-means
    num_clusters = 30
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(filtered_points_np)
    labels = kmeans.labels_

    # Присвоение цветов кластерам
    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

    # Создание облака точек с цветами кластеров
    kmeans_pcd = o3d.geometry.PointCloud()
    kmeans_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)
    kmeans_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

    # Создание ограничивающего параллелепипеда для каждого кластера
    bbox_list = []
    for label in np.unique(labels):
        if label == -1:
            continue
        cluster_points = filtered_points_np[labels == label]
        bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster_points))
        bbox.color = (0, 0, 0)
        bbox_list.append(bbox)

    # Создание линий от красной точки к центрам кластеров
    lines = []
    for cluster_center in kmeans.cluster_centers_:
        line = o3d.geometry.LineSet()
        line.points = o3d.utility.Vector3dVector([center, cluster_center])
        line.lines = o3d.utility.Vector2iVector([[0, 1]])
        line.colors = o3d.utility.Vector3dVector([[0, 0, 0]])
        lines.append(line)

    # Визуализация облака точек с кластеризацией K-means, ограничивающих параллелепипедов, центром и линиями
    o3d.visualization.draw_geometries([kmeans_pcd, center_point] + bbox_list + lines)


In [71]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Заменяем переменную velodyne_file на filtered_points_np
points_np = filtered_points_np  # filtered_points_np должна быть определена заранее

# Проверка наличия данных в облаке точек
if len(points_np) == 0:
    print("Облако точек пустое.")
else:
    # Масштабирование данных
    scaler = StandardScaler()
    scaled_points_np = scaler.fit_transform(points_np)

    # Отсечение точек снизу по оси Z
    cutting_height = -1.2
    cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
    cut_points_np = points_np[cut_indices]
    cut_scaled_points_np = scaled_points_np[cut_indices]

    # Определение числа ближайших соседей
    n_neighbors = 1

    # Поиск ближайших соседей для каждой точки
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
    distances, indices = nbrs.kneighbors(cut_scaled_points_np)

    # Вычисление среднего расстояния до n_neighbors точек для каждой точки
    avg_distances = np.mean(distances, axis=1)

    # Определение порогового значения для удаления редких точек
    threshold_low = np.percentile(avg_distances, 1)

    # Отфильтровать точки по пороговому значению
    filtered_indices = np.where(avg_distances <= threshold_low)[0]
    filtered_points_np = cut_points_np[filtered_indices]

    # Вычисление центра масс облака точек
    center = np.mean(filtered_points_np, axis=0)

    # Создание красной точки в центре облака точек
    center_point = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)
    center_point.paint_uniform_color([1.0, 0.0, 0.0])
    center_point.translate(center)

    # Выполнение кластеризации K-means
    num_clusters = 30
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(filtered_points_np)
    labels = kmeans.labels_

    # Создание линий от красной точки к центрам кластеров
    lines = []
    for cluster_center in kmeans.cluster_centers_:
        line = o3d.geometry.LineSet()
        line.points = o3d.utility.Vector3dVector([center, cluster_center])
        line.lines = o3d.utility.Vector2iVector([[0, 1]])
        line.colors = o3d.utility.Vector3dVector([[0, 0, 0]])
        lines.append(line)

    # Визуализация линий от красной точки к центрам кластеров
    o3d.visualization.draw_geometries([center_point] + lines)


# PART 2

In [72]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Заменяем переменную velodyne_file на filtered_points_np
points_np = filtered_points_np  # filtered_points_np должна быть определена заранее

# Проверка наличия данных в облаке точек
if len(points_np) == 0:
    print("Облако точек пустое.")
else:
    # Масштабирование данных
    scaler = StandardScaler()
    scaled_points_np = scaler.fit_transform(points_np)

    # Отсечение точек снизу по оси Z
    cutting_height = -1.2
    cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
    cut_points_np = points_np[cut_indices]
    cut_scaled_points_np = scaled_points_np[cut_indices]

    # Определение числа ближайших соседей
    n_neighbors = 1

    # Поиск ближайших соседей для каждой точки
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
    distances, indices = nbrs.kneighbors(cut_scaled_points_np)

    # Вычисление среднего расстояния до n_neighbors точек для каждой точки
    avg_distances = np.mean(distances, axis=1)

    # Определение порогового значения для удаления редких точек
    threshold_low = np.percentile(avg_distances, 1)

    # Отфильтровать точки по пороговому значению
    filtered_indices = np.where(avg_distances <= threshold_low)[0]
    filtered_points_np = cut_points_np[filtered_indices]

    # Вычисление центра масс облака точек
    center = np.mean(filtered_points_np, axis=0)

    # Создание красной точки в центре облака точек
    center_point = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)
    center_point.paint_uniform_color([1.0, 0.0, 0.0])
    center_point.translate(center)

    # Выполнение кластеризации K-means
    num_clusters = 30
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(filtered_points_np)
    labels = kmeans.labels_

    # Присвоение цветов кластерам
    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

    # Создание облака точек с цветами кластеров
    kmeans_pcd = o3d.geometry.PointCloud()
    kmeans_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)
    kmeans_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

    # Создание ограничивающего параллелепипеда для каждого кластера
    bbox_list = []
    cluster_sizes = []
    for label in np.unique(labels):
        if label == -1:
            continue
        cluster_points = filtered_points_np[labels == label]
        bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster_points))
        bbox.color = (0, 0, 0)
        bbox_list.append(bbox)
        cluster_sizes.append(len(cluster_points))

    # Создание линий от красной точки к центрам кластеров
    lines = []
    for cluster_center in kmeans.cluster_centers_:
        line = o3d.geometry.LineSet()
        line.points = o3d.utility.Vector3dVector([center, cluster_center])
        line.lines = o3d.utility.Vector2iVector([[0, 1]])
        line.colors = o3d.utility.Vector3dVector([[0, 0, 0]])
        lines.append(line)

    # Визуализация облака точек с кластеризацией K-means, ограничивающих параллелепипедов, центром и линиями
    o3d.visualization.draw_geometries([kmeans_pcd, center_point] + bbox_list + lines)

    # Вывод информации о каждом ограничивающем параллелепипеде
    for i, bbox in enumerate(bbox_list, 1):
        print(f"Ограничивающий параллелепипед {i}:")
        print(f"Центр: {bbox.get_center()}")
        print(f"Размеры: {bbox.get_extent()}")
        print(f"Количество точек в кластере: {cluster_sizes[i-1]}")


Ограничивающий параллелепипед 1:
Центр: [ 7.46649981  1.89899997 -0.63149998]
Размеры: [4.56099987 3.72999994 1.13299996]
Количество точек в кластере: 1386
Ограничивающий параллелепипед 2:
Центр: [-18.39800024  -4.20750004  -0.19500002]
Размеры: [6.31400013 4.64700019 1.99000007]
Количество точек в кластере: 1183
Ограничивающий параллелепипед 3:
Центр: [ 2.79950005 -6.99900007 -0.34399998]
Размеры: [3.07099998 5.71600056 1.70799994]
Количество точек в кластере: 4520
Ограничивающий параллелепипед 4:
Центр: [-0.40350002  7.55250001 -0.388     ]
Размеры: [2.71899998 2.66900015 1.62200001]
Количество точек в кластере: 1532
Ограничивающий параллелепипед 5:
Центр: [-15.17199945  11.21650028  -0.56949998]
Размеры: [5.56199932 4.95100021 1.25699995]
Количество точек в кластере: 673
Ограничивающий параллелепипед 6:
Центр: [-2.17029991e+01 -1.19740000e+01 -2.10000277e-02]
Размеры: [7.72200012 1.74399948 2.352     ]
Количество точек в кластере: 642
Ограничивающий параллелепипед 7:
Центр: [20.0549

In [73]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
# import os

# Заменяем переменную velodyne_file на filtered_points_np
points_np = filtered_points_np  # filtered_points_np должна быть определена заранее

# Проверка наличия данных в облаке точек
if len(points_np) == 0:
    print("Облако точек пустое.")
else:
    # Масштабирование данных
    scaler = StandardScaler()
    scaled_points_np = scaler.fit_transform(points_np)

    # Отсечение точек снизу по оси Z
    cutting_height = -1.2
    cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
    cut_points_np = points_np[cut_indices]
    cut_scaled_points_np = scaled_points_np[cut_indices]

    # Определение числа ближайших соседей
    n_neighbors = 1

    # Поиск ближайших соседей для каждой точки
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
    distances, indices = nbrs.kneighbors(cut_scaled_points_np)

    # Вычисление среднего расстояния до n_neighbors точек для каждой точки
    avg_distances = np.mean(distances, axis=1)

    # Определение порогового значения для удаления редких точек
    threshold_low = np.percentile(avg_distances, 1)

    # Отфильтровать точки по пороговому значению
    filtered_indices = np.where(avg_distances <= threshold_low)[0]
    filtered_points_np = cut_points_np[filtered_indices]

    # Вычисление центра масс облака точек
    center = np.mean(filtered_points_np, axis=0)

    # Создание красной точки в центре облака точек
    center_point = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)
    center_point.paint_uniform_color([1.0, 0.0, 0.0])
    center_point.translate(center)

    # Выполнение кластеризации K-means
    num_clusters = 30
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(filtered_points_np)
    labels = kmeans.labels_

    # Присвоение цветов кластерам
    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

    # Создание облака точек с цветами кластеров
    kmeans_pcd = o3d.geometry.PointCloud()
    kmeans_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)
    kmeans_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

    # Создание ограничивающего параллелепипеда для каждого кластера
    bbox_list = []
    cluster_sizes = []
    for label in np.unique(labels):
        if label == -1:
            continue
        cluster_points = filtered_points_np[labels == label]
        bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster_points))
        bbox.color = (0, 0, 0)
        bbox_list.append(bbox)
        cluster_sizes.append(len(cluster_points))

    # Создание линий от красной точки к центрам кластеров
    lines = []
    for cluster_center in kmeans.cluster_centers_:
        line = o3d.geometry.LineSet()
        line.points = o3d.utility.Vector3dVector([center, cluster_center])
        line.lines = o3d.utility.Vector2iVector([[0, 1]])
        line.colors = o3d.utility.Vector3dVector([[0, 0, 0]])
        lines.append(line)

    # Визуализация облака точек с кластеризацией K-means, ограничивающих параллелепипедов, центром и линиями
    o3d.visualization.draw_geometries([kmeans_pcd, center_point] + bbox_list + lines)

    # # Вывод информации о каждом ограничивающем параллелепипеде и сохранение в файл
    # output_file_path = "bbox_info.txt"
    # with open(output_file_path, "w", encoding="utf-8") as file:
    #     for i, bbox in enumerate(bbox_list, 1):
    #         file.write(f"Ограничивающий параллелепипед {i}:\n")
    #         file.write(f"Центр: {bbox.get_center().tolist()}\n")
    #         file.write(f"Размеры: {bbox.get_extent().tolist()}\n")
    #         file.write(f"Количество точек в кластере: {cluster_sizes[i-1]}\n")

    #         print(f"Ограничивающий параллелепипед {i}:")
    #         print(f"Центр: {bbox.get_center()}")
    #         print(f"Размеры: {bbox.get_extent()}")
    #         print(f"Количество точек в кластере: {cluster_sizes[i-1]}")

    # print(f"Информация о каждом ограничивающем параллелепипеде сохранена в файл: {output_file_path}")


Ограничивающий параллелепипед 1:
Центр: [ 7.46649981  1.89899997 -0.63149998]
Размеры: [4.56099987 3.72999994 1.13299996]
Количество точек в кластере: 1386
Ограничивающий параллелепипед 2:
Центр: [-18.39800024  -4.20750004  -0.19500002]
Размеры: [6.31400013 4.64700019 1.99000007]
Количество точек в кластере: 1183
Ограничивающий параллелепипед 3:
Центр: [ 2.79950005 -6.99900007 -0.34399998]
Размеры: [3.07099998 5.71600056 1.70799994]
Количество точек в кластере: 4520
Ограничивающий параллелепипед 4:
Центр: [-0.40350002  7.55250001 -0.388     ]
Размеры: [2.71899998 2.66900015 1.62200001]
Количество точек в кластере: 1532
Ограничивающий параллелепипед 5:
Центр: [-15.17199945  11.21650028  -0.56949998]
Размеры: [5.56199932 4.95100021 1.25699995]
Количество точек в кластере: 673
Ограничивающий параллелепипед 6:
Центр: [-2.17029991e+01 -1.19740000e+01 -2.10000277e-02]
Размеры: [7.72200012 1.74399948 2.352     ]
Количество точек в кластере: 642
Ограничивающий параллелепипед 7:
Центр: [20.0549

In [78]:
import numpy as np
import os

# Вывод центра кластера и расстояния до центра координат
for i, center_coord in enumerate(kmeans.cluster_centers_):
    distance_to_center = np.linalg.norm(center_coord)
    print(f"Центр кластера {i}: {center_coord}, Расстояние до центра координат: {distance_to_center}")

# Путь для сохранения файла
output_dir = r'D:\Cluster_proj\VIPCClustering\KITTI_clusters\experiment_000008\cluster'

# Создание директории, если она не существует
os.makedirs(output_dir, exist_ok=True)

# Имя файла для сохранения информации
output_file_path = os.path.join(output_dir, "bbox_info.txt")

# Запись информации в файл
with open(output_file_path, "w", encoding="utf-8") as file:
    for i, bbox in enumerate(bbox_list, 1):
        file.write(f"Ограничивающий параллелепипед {i}:\n")
        file.write(f"Центр: {bbox.get_center().tolist()}\n")
        file.write(f"Размеры: {bbox.get_extent().tolist()}\n")
        file.write(f"Количество точек в кластере: {cluster_sizes[i-1]}\n")
        file.write(f"Расстояние до центра координат: {np.linalg.norm(bbox.get_center())}\n")

        print(f"Ограничивающий параллелепипед {i}:")
        print(f"Центр: {bbox.get_center()}")
        print(f"Размеры: {bbox.get_extent()}")
        print(f"Количество точек в кластере: {cluster_sizes[i-1]}")
        print(f"Расстояние до центра координат: {np.linalg.norm(bbox.get_center())}")

print(f"Информация о каждом ограничивающем параллелепипеде сохранена в файл: {output_file_path}")


Центр кластера 0: [ 7.38157     1.1833284  -0.75219214], Расстояние до центра координат: 7.513563632965088
Центр кластера 1: [-17.919664    -4.2885084   -0.31951305], Расстояние до центра координат: 18.428449630737305
Центр кластера 2: [ 2.7432063 -5.6483665 -0.3102945], Расстояние до центра координат: 6.28693151473999
Центр кластера 3: [-0.4773799   6.957901   -0.73464036], Расстояние до центра координат: 7.012843608856201
Центр кластера 4: [-15.490846   10.14682    -0.6932348], Расстояние до центра координат: 18.531185150146484
Центр кластера 5: [-1.9621719e+01 -1.2198488e+01  6.3614249e-03], Расстояние до центра координат: 23.10443687438965
Центр кластера 6: [20.33069     1.0635942   0.32077065], Расстояние до центра координат: 20.361019134521484
Центр кластера 7: [-11.0508795 -15.560546   -0.6521246], Расстояние до центра координат: 19.096538543701172
Центр кластера 8: [ 3.091351    2.4098222  -0.71416163], Расстояние до центра координат: 3.9841837882995605
Центр кластера 9: [ 5.53

# PART 3

In [79]:
# import open3d as o3d
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler
# from sklearn.neighbors import NearestNeighbors
# from sklearn.metrics import silhouette_score
# from scipy.spatial.distance import pdist

# def dunn_index(X, labels):
#     # Check if the number of data points matches the number of labels
#     if len(X) != len(labels):
#         raise ValueError("Number of data points and labels do not match.")

#     # Check if there are enough unique labels to form clusters
#     unique_labels = np.unique(labels)
#     if len(unique_labels) < 2:
#         raise ValueError("At least two different clusters are required.")

#     # Compute intra-cluster distances
#     intra_cluster_distances = []
#     for label in unique_labels:
#         cluster_points = X[labels == label]
#         if len(cluster_points) <= 1:
#             intra_cluster_distances.append(0)  # Not enough data to assess intra-cluster distances
#         else:
#             intra_cluster_distances.append(np.max(pdist(cluster_points)))

#     # Compute inter-cluster distances
#     min_inter_cluster_distance = np.inf
#     for i in range(len(unique_labels)):
#         for j in range(i + 1, len(unique_labels)):
#             cluster_points_i = X[labels == unique_labels[i]]
#             cluster_points_j = X[labels == unique_labels[j]]
#             inter_cluster_distance = np.min(pdist(np.vstack([cluster_points_i, cluster_points_j])))
#             if inter_cluster_distance < min_inter_cluster_distance:
#                 min_inter_cluster_distance = inter_cluster_distance

#     # Compute Dunn Index
#     dunn_index_value = min_inter_cluster_distance / np.max(intra_cluster_distances)
#     return dunn_index_value

# # Заменяем переменную velodyne_file на filtered_points_np
# points_np = filtered_points_np  # filtered_points_np должна быть определена заранее

# # Проверка наличия данных в облаке точек
# if len(points_np) == 0:
#     print("Облако точек пустое.")
# else:
#     # Масштабирование данных
#     scaler = StandardScaler()
#     scaled_points_np = scaler.fit_transform(points_np)

#     # Отсечение точек снизу по оси Z
#     cutting_height = -1.2
#     cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
#     cut_points_np = points_np[cut_indices]
#     cut_scaled_points_np = scaled_points_np[cut_indices]

#     # Определение числа ближайших соседей
#     n_neighbors = 1

#     # Поиск ближайших соседей для каждой точки
#     nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
#     distances, indices = nbrs.kneighbors(cut_scaled_points_np)

#     # Вычисление среднего расстояния до n_neighbors точек для каждой точки
#     avg_distances = np.mean(distances, axis=1)

#     # Определение порогового значения для удаления редких точек
#     threshold_low = np.percentile(avg_distances, 1)

#     # Отфильтровать точки по пороговому значению
#     filtered_indices = np.where(avg_distances <= threshold_low)[0]
#     filtered_points_np = cut_points_np[filtered_indices]

#     # Вычисление центра масс облака точек
#     center = np.mean(filtered_points_np, axis=0)

#     # Создание красной точки в центре облака точек
#     center_point = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)
#     center_point.paint_uniform_color([1.0, 0.0, 0.0])
#     center_point.translate(center)

#     # Выполнение кластеризации K-means
#     num_clusters = 30
#     kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(filtered_points_np)
#     labels = kmeans.labels_

#     # Вычисление Silhouette Score
#     silhouette_score_value = silhouette_score(filtered_points_np, labels)
#     print("Silhouette Score:", silhouette_score_value)

#     # Вычисление Dunn Index
#     try:
#         dunn_index_value = dunn_index(filtered_points_np, labels)
#         print("Dunn Index:", dunn_index_value)
#     except ValueError as e:
#         print("Error:", e)

#     # Присвоение цветов кластерам
#     max_label = labels.max()
#     colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

#     # Создание облака точек с цветами кластеров
#     kmeans_pcd = o3d.geometry.PointCloud()
#     kmeans_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)
#     kmeans_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

#     # Создание ограничивающего параллелепипеда для каждого кластера
#     bbox_list = []
#     for label in np.unique(labels):
#         if label == -1:
#             continue
#         cluster_points = filtered_points_np[labels == label]
#         bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster_points))
#         bbox.color = (0, 0, 0)
#         bbox_list.append(bbox)

#     # Создание линий от красной точки к центрам кластеров
#     lines = []
#     for cluster_center in kmeans.cluster_centers_:
#         line = o3d.geometry.LineSet()
#         line.points = o3d.utility.Vector3dVector([center, cluster_center])
#         line.lines = o3d.utility.Vector2iVector([[0, 1]])
#         line.colors = o3d.utility.Vector3dVector([[0, 0, 0]])
#         lines.append(line)

#     # Визуализация облака точек с кластеризацией K-means, ограничивающих параллепипедов, центром и линиями
#     o3d.visualization.draw_geometries([kmeans_pcd, center_point] + bbox_list + lines)


Silhouette Score: 0.51736397
Dunn Index: 0.0006200075893502445


In [80]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.spatial.distance import pdist

def dunn_index(X, labels):
    # Check if the number of data points matches the number of labels
    if len(X) != len(labels):
        raise ValueError("Number of data points and labels do not match.")

    # Check if there are enough unique labels to form clusters
    unique_labels = np.unique(labels)
    if len(unique_labels) < 2:
        raise ValueError("At least two different clusters are required.")

    # Compute intra-cluster distances
    intra_cluster_distances = []
    for label in unique_labels:
        cluster_points = X[labels == label]
        if len(cluster_points) <= 1:
            intra_cluster_distances.append(0)  # Not enough data to assess intra-cluster distances
        else:
            intra_cluster_distances.append(np.max(pdist(cluster_points)))

    # Compute inter-cluster distances
    min_inter_cluster_distance = np.inf
    for i in range(len(unique_labels)):
        for j in range(i + 1, len(unique_labels)):
            cluster_points_i = X[labels == unique_labels[i]]
            cluster_points_j = X[labels == unique_labels[j]]
            inter_cluster_distance = np.min(pdist(np.vstack([cluster_points_i, cluster_points_j])))
            if inter_cluster_distance < min_inter_cluster_distance:
                min_inter_cluster_distance = inter_cluster_distance

    # Compute Dunn Index
    dunn_index_value = min_inter_cluster_distance / np.max(intra_cluster_distances)
    return dunn_index_value

# Заменяем переменную velodyne_file на filtered_points_np
points_np = filtered_points_np  # filtered_points_np должна быть определена заранее

# Проверка наличия данных в облаке точек
if len(points_np) == 0:
    print("Облако точек пустое.")
else:
    # Масштабирование данных
    scaler = StandardScaler()
    scaled_points_np = scaler.fit_transform(points_np)

    # Отсечение точек снизу по оси Z
    cutting_height = -1.2
    cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
    cut_points_np = points_np[cut_indices]
    cut_scaled_points_np = scaled_points_np[cut_indices]

    # Определение числа ближайших соседей
    n_neighbors = 1

    # Поиск ближайших соседей для каждой точки
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
    distances, indices = nbrs.kneighbors(cut_scaled_points_np)

    # Вычисление среднего расстояния до n_neighbors точек для каждой точки
    avg_distances = np.mean(distances, axis=1)

    # Определение порогового значения для удаления редких точек
    threshold_low = np.percentile(avg_distances, 1)

    # Отфильтровать точки по пороговому значению
    filtered_indices = np.where(avg_distances <= threshold_low)[0]
    filtered_points_np = cut_points_np[filtered_indices]

    # Вычисление центра масс облака точек
    center = np.mean(filtered_points_np, axis=0)

    # Создание красной точки в центре облака точек
    center_point = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)
    center_point.paint_uniform_color([1.0, 0.0, 0.0])
    center_point.translate(center)

    # Выполнение кластеризации K-means
    num_clusters = 30
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(filtered_points_np)
    labels = kmeans.labels_

    # Вычисление Silhouette Score
    silhouette_score_value = silhouette_score(filtered_points_np, labels)
    print("Silhouette Score:", silhouette_score_value)

    # Вычисление Dunn Index
    try:
        dunn_index_value = dunn_index(filtered_points_np, labels)
        print("Dunn Index:", dunn_index_value)
    except ValueError as e:
        print("Error:", e)

    # Вычисление Davies-Bouldin Index
    davies_bouldin_value = davies_bouldin_score(filtered_points_np, labels)
    print("Davies-Bouldin Index:", davies_bouldin_value)

    # Вычисление Calinski-Harabasz Index
    calinski_harabasz_value = calinski_harabasz_score(filtered_points_np, labels)
    print("Calinski–Harabasz Index:", calinski_harabasz_value)
    # Вычисление Calinski-Harabasz Index
    calinski_harabasz_value = calinski_harabasz_score(filtered_points_np, labels)
    print("Calinski–Harabasz Index:", calinski_harabasz_value)

    # Вычисление C-index (Calinski-Harabasz Index)
    c_index = calinski_harabasz_value / davies_bouldin_value
    print("C-index:", c_index)


    # Присвоение цветов кластерам
    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

    # Создание облака точек с цветами кластеров
    kmeans_pcd = o3d.geometry.PointCloud()
    kmeans_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)
    kmeans_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

    # Создание ограничивающего параллелепипеда для каждого кластера
    bbox_list = []
    for label in np.unique(labels):
        if label == -1:
            continue
        cluster_points = filtered_points_np[labels == label]
        bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster_points))
        bbox.color = (0, 0, 0)
        bbox_list.append(bbox)

    # Создание линий от красной точки к центрам кластеров
    lines = []
    for cluster_center in kmeans.cluster_centers_:
        line = o3d.geometry.LineSet()
        line.points = o3d.utility.Vector3dVector([center, cluster_center])
        line.lines = o3d.utility.Vector2iVector([[0, 1]])
        line.colors = o3d.utility.Vector3dVector([[0, 0, 0]])
        lines.append(line)

    # Визуализация облака точек с кластеризацией K-means, ограничивающих параллепипедов, центром и линиями
    o3d.visualization.draw_geometries([kmeans_pcd, center_point] + bbox_list + lines)



Silhouette Score: 0.51736397
Dunn Index: 0.0006200075893502445
Davies-Bouldin Index: 0.6702602815651734
Calinski–Harabasz Index: 114922.60749871065
Calinski–Harabasz Index: 114922.60749871065
C-index: 171459.67120466477


In [81]:
import os

# Путь для сохранения файла
output_dir = r'D:\Cluster_proj\VIPCClustering\KITTI_clusters\experiment_000008\cluster'

# Создание директории, если она не существует
os.makedirs(output_dir, exist_ok=True)

# Имя файла для сохранения информации
output_file_path = os.path.join(output_dir, "indexes.txt")

# Запись информации в файл
with open(output_file_path, "w", encoding="utf-8") as file:
    file.write(f"Silhouette Score: {silhouette_score_value}\n")
    file.write(f"Dunn Index: {dunn_index_value}\n")
    file.write(f"Davies-Bouldin Index: {davies_bouldin_value}\n")
    file.write(f"Calinski–Harabasz Index: {calinski_harabasz_value}\n")
    file.write(f"C-index: {c_index}\n")

print(f"Индексы сохранены в файл: {output_file_path}")


Индексы сохранены в файл: D:\Cluster_proj\VIPCClustering\KITTI_clusters\experiment_000008\cluster\indexes.txt


In [ ]:
# import sys
# import os

# # Путь к директории, где будет создаваться файл
# output_dir = 'D:/Cluster_proj/VIPCClustering/KITTI_clusters/experiment_000008/cluster'

# # Создание директории, если она не существует
# os.makedirs(output_dir, exist_ok=True)

# # Формирование пути к файлу вывода
# output_file_path = os.path.join(output_dir, "output.txt")

# # Открытие файла для записи
# with open(output_file_path, "w") as file:
#     # Запись номера кластера и расстояния до центра кластера в файл
#     for i, (cluster_center, label) in enumerate(zip(kmeans.cluster_centers_, labels)):
#         distances_to_center = np.linalg.norm(filtered_points_np[labels == label] - cluster_center, axis=1)  # Расстояния до центра кластера
#         for distance in distances_to_center:
#             line = f"cluster number {label} ; distance - {distance}\n"
#             file.write(line)

# print(f"Результаты сохранены в файл: {output_file_path}")


Результаты сохранены в файл: D:/Cluster_proj/VIPCClustering/KITTI_clusters/experiment_000008/cluster\output.txt


# PART 2231245124

(50235, 4)
float64


In [ ]:
# import open3d as o3d
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler
# from sklearn.neighbors import NearestNeighbors
# from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
# from scipy.spatial.distance import pdist
# import os


# # print(colors.shape)
# # print(colors.dtype)
# # Преобразование из RGBA в RGB
# colors_rgb = colors[:, :3]


# def dunn_index(X, labels):
#     # Check if the number of data points matches the number of labels
#     if len(X) != len(labels):
#         raise ValueError("Number of data points and labels do not match.")

#     # Check if there are enough unique labels to form clusters
#     unique_labels = np.unique(labels)
#     if len(unique_labels) < 2:
#         raise ValueError("At least two different clusters are required.")

#     # Compute intra-cluster distances
#     intra_cluster_distances = []
#     for label in unique_labels:
#         cluster_points = X[labels == label]
#         if len(cluster_points) <= 1:
#             intra_cluster_distances.append(0)  # Not enough data to assess intra-cluster distances
#         else:
#             intra_cluster_distances.append(np.max(pdist(cluster_points)))

#     # Compute inter-cluster distances
#     min_inter_cluster_distance = np.inf
#     for i in range(len(unique_labels)):
#         for j in range(i + 1, len(unique_labels)):
#             cluster_points_i = X[labels == unique_labels[i]]
#             cluster_points_j = X[labels == unique_labels[j]]
#             inter_cluster_distance = np.min(pdist(np.vstack([cluster_points_i, cluster_points_j])))
#             if inter_cluster_distance < min_inter_cluster_distance:
#                 min_inter_cluster_distance = inter_cluster_distance

#     # Compute Dunn Index
#     dunn_index_value = min_inter_cluster_distance / np.max(intra_cluster_distances)
#     return dunn_index_value

# # Проверка наличия данных в облаке точек
# if len(points_np) == 0:
#     print("Облако точек пустое.")
# else:
#     # Масштабирование данных
#     scaler = StandardScaler()
#     scaled_points_np = scaler.fit_transform(points_np)

#     # Отсечение точек снизу по оси Z
#     cutting_height = -1.2
#     cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
#     cut_points_np = points_np[cut_indices]
#     cut_scaled_points_np = scaled_points_np[cut_indices]

#     # Определение числа ближайших соседей
#     n_neighbors = 1

#     # Поиск ближайших соседей для каждой точки
#     nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
#     distances, indices = nbrs.kneighbors(cut_scaled_points_np)

#     # Вычисление среднего расстояния до n_neighbors точек для каждой точки
#     avg_distances = np.mean(distances, axis=1)

#     # Определение порогового значения для удаления редких точек
#     threshold_low = np.percentile(avg_distances, 1)

#     # Отфильтровать точки по пороговому значению
#     filtered_indices = np.where(avg_distances <= threshold_low)[0]
#     filtered_points_np = cut_points_np[filtered_indices]

#     # Выполнение кластеризации K-means
#     num_clusters = 30
#     kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(filtered_points_np)
#     labels = kmeans.labels_

#     # Вычисление Silhouette Score
#     silhouette_score_value = silhouette_score(filtered_points_np, labels)
#     print("Silhouette Score:", silhouette_score_value)

#     # Вычисление Dunn Index
#     try:
#         dunn_index_value = dunn_index(filtered_points_np, labels)
#         print("Dunn Index:", dunn_index_value)
#     except ValueError as e:
#         print("Error:", e)

#     # Вычисление Davies-Bouldin Index
#     davies_bouldin_value = davies_bouldin_score(filtered_points_np, labels)
#     print("Davies-Bouldin Index:", davies_bouldin_value)

#     # Вычисление Calinski-Harabasz Index
#     calinski_harabasz_value = calinski_harabasz_score(filtered_points_np, labels)
#     print("Calinski–Harabasz Index:", calinski_harabasz_value)
#     # Вычисление Calinski-Harabasz Index
#     calinski_harabasz_value = calinski_harabasz_score(filtered_points_np, labels)
#     print("Calinski–Harabasz Index:", calinski_harabasz_value)

#     # Вычисление C-index (Calinski-Harabasz Index)
#     c_index = calinski_harabasz_value / davies_bouldin_value
#     print("C-index:", c_index)

#     # Присвоение цветов кластерам
#     max_label = labels.max()
#     colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

#     # Создание облака точек с цветами кластеров
#     kmeans_pcd = o3d.geometry.PointCloud()
#     kmeans_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)
#     kmeans_pcd.colors = o3d.utility.Vector3dVector(colors_rgb)




#     # Создание ограничивающего параллелепипеда для каждого кластера
#     bbox_list = []
#     for label in np.unique(labels):
#         if label == -1:
#             continue
#         cluster_points = filtered_points_np[labels == label]
#         bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster_points))
#         bbox.color = (0, 0, 0)
#         bbox_list.append(bbox)

#     # Создание линий от красной точки к центрам кластеров
#     lines = []
#     for cluster_center in kmeans.cluster_centers_:
#         line = o3d.geometry.LineSet()
#         line.points = o3d.utility.Vector3dVector([center, cluster_center])
#         line.lines = o3d.utility.Vector2iVector([[0, 1]])
#         line.colors = o3d.utility.Vector3dVector([[0, 0, 0]])
#         lines.append(line)

#     # Визуализация облака точек с кластеризацией K-means, ограничивающих параллепипедов, центром и линиями
#     o3d.visualization.draw_geometries([kmeans_pcd, center_point] + bbox_list + lines)

#     # Путь к директории, где будет создаваться файл
#     output_dir = 'D:/Cluster_proj/VIPCClustering/KITTI_clusters/experiment_000008/cluster'

#     # Создание директории, если она не существует
#     os.makedirs(output_dir, exist_ok=True)

#     # Формирование пути к файлу вывода
#     output_file_path = os.path.join(output_dir, "output.txt")

#     # Открытие файла для записи
#     with open(output_file_path, "w") as file:
#         # Запись номера кластера и расстояния до центра кластера в файл
#         for i, cluster_center in enumerate(kmeans.cluster_centers_):
#             distances_to_center = np.linalg.norm(filtered_points_np[labels == i] - cluster_center, axis=1)
#             for distance in distances_to_center:
#                 line = f"cluster number {i} ; distance - {distance}\n"
#                 file.write(line)

#     print(f"Результаты сохранены в файл: {output_file_path}")



Silhouette Score: 0.51736397
Dunn Index: 0.0006200075893502445
Davies-Bouldin Index: 0.6702602815651734
Calinski–Harabasz Index: 114922.60749871065
Calinski–Harabasz Index: 114922.60749871065
C-index: 171459.67120466477
Результаты сохранены в файл: D:/Cluster_proj/VIPCClustering/KITTI_clusters/experiment_000008/cluster\output.txt


# OTHER

In [ ]:
# import open3d as o3d
# import numpy as np
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler
# from sklearn.neighbors import NearestNeighbors
# from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
# from scipy.spatial.distance import pdist

# def dunn_index(X, labels):
#     # Check if the number of data points matches the number of labels
#     if len(X) != len(labels):
#         raise ValueError("Number of data points and labels do not match.")

#     # Check if there are enough unique labels to form clusters
#     unique_labels = np.unique(labels)
#     if len(unique_labels) < 2:
#         raise ValueError("At least two different clusters are required.")

#     # Compute intra-cluster distances
#     intra_cluster_distances = []
#     for label in unique_labels:
#         cluster_points = X[labels == label]
#         if len(cluster_points) <= 1:
#             intra_cluster_distances.append(0)  # Not enough data to assess intra-cluster distances
#         else:
#             intra_cluster_distances.append(np.max(pdist(cluster_points)))

#     # Compute inter-cluster distances
#     min_inter_cluster_distance = np.inf
#     for i in range(len(unique_labels)):
#         for j in range(i + 1, len(unique_labels)):
#             cluster_points_i = X[labels == unique_labels[i]]
#             cluster_points_j = X[labels == unique_labels[j]]
#             inter_cluster_distance = np.min(pdist(np.vstack([cluster_points_i, cluster_points_j])))
#             if inter_cluster_distance < min_inter_cluster_distance:
#                 min_inter_cluster_distance = inter_cluster_distance

#     # Compute Dunn Index
#     dunn_index_value = min_inter_cluster_distance / np.max(intra_cluster_distances)
#     return dunn_index_value

# # Проверка наличия данных в облаке точек
# if len(filtered_points_np) == 0:
#     print("Облако точек пустое.")
# else:
#     # Масштабирование данных
#     scaler = StandardScaler()
#     scaled_points_np = scaler.fit_transform(filtered_points_np)

#     # Выполнение кластеризации K-means
#     num_clusters = 30
#     kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(filtered_points_np)
#     labels = kmeans.labels_

#     # Вычисление Silhouette Score
#     silhouette_score_value = silhouette_score(filtered_points_np, labels)
#     print("Silhouette Score:", silhouette_score_value)

#     # Вычисление Dunn Index
#     try:
#         dunn_index_value = dunn_index(filtered_points_np, labels)
#         print("Dunn Index:", dunn_index_value)
#     except ValueError as e:
#         print("Error:", e)

#     # Вычисление Davies-Bouldin Index
#     davies_bouldin_value = davies_bouldin_score(filtered_points_np, labels)
#     print("Davies-Bouldin Index:", davies_bouldin_value)

#     # Вычисление Calinski-Harabasz Index
#     calinski_harabasz_value = calinski_harabasz_score(filtered_points_np, labels)
#     print("Calinski–Harabasz Index:", calinski_harabasz_value)

#     # Вычисление C-index (Calinski-Harabasz Index)
#     c_index = calinski_harabasz_value / davies_bouldin_value
#     print("C-index:", c_index)

#     # Присвоение цветов кластерам
#     max_label = labels.max()
#     colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

#     # Создание облака точек с цветами кластеров
#     kmeans_pcd = o3d.geometry.PointCloud()
#     kmeans_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)
#     kmeans_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

#     # Создание ограничивающих параллелепипедов для каждого кластера с номерами кластеров
#     bbox_list = []
#     for label in np.unique(labels):
#         if label == -1:
#             continue
#         cluster_points = filtered_points_np[labels == label]
#         bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster_points))
#         bbox.color = (0, 0, 0)
#         bbox_list.append(bbox)

#     # Визуализация с ограничивающими параллелепипедами и текстом
#     vis = o3d.visualization.Visualizer()
#     vis.create_window()

#     for geometry in [kmeans_pcd] + bbox_list:
#         vis.add_geometry(geometry)

#     vis.run()
#     vis.destroy_window()


Silhouette Score: 0.51736397
Dunn Index: 0.0006200075893502445
Davies-Bouldin Index: 0.6702602815651734
Calinski–Harabasz Index: 114922.60749871065
Calinski–Harabasz Index: 114922.60749871065
C-index: 171459.67120466477
[Open3D WARNING] Read utility::IJsonConvertible failed: unknown file extension.


AttributeError: module 'open3d.visualization' has no attribute 'create_text_2d'